In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request as urllib2

Список ссылок

In [2]:
list_of_URL = ['https://makeupstore.ru/product/394224/reviews/',
               'https://makeupstore.ru/product/599/reviews/',
               'https://makeupstore.ru/product/15499/reviews/',
               'https://makeupstore.ru/product/193337/reviews/',
               'https://makeupstore.ru/product/301777/reviews/',
               'https://makeupstore.ru/product/3173/reviews/',
               'https://makeupstore.ru/product/4146/reviews/',
               'https://makeupstore.ru/product/4176/reviews/'
              ]

In [3]:
data = []
for pg in list_of_URL:
    page = urllib2.urlopen (pg)
    soup = BeautifulSoup (page, 'html.parser')
    quotes1 = soup.find('p', itemprop='reviewBody')
    price = quotes1.text
    data.append(price)

Сохраяем вытащенные отзывы в файл

In [4]:
MyFile=open('output.txt', 'w', encoding = 'utf-8')
for element in data:
    MyFile.write(element)
    MyFile.write('\n')
MyFile.close()

После вручную отделяю позитивные и негативные отзывы в два файла. 

Открываю текст с позитивными и негативными отзывами

In [5]:
with open ('positive.txt', 'r', encoding = 'utf-8') as pos:
    text = pos.read()

In [6]:
with open ('negative.txt', 'r') as neg:
    text_1 = neg.read()

В файле smth.txt находятся 10 отзывов

In [7]:
with open ('smth.txt', 'r', encoding = 'utf-8') as smth:
    text_2 = smth.readlines()

In [8]:
import pymorphy2
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter
morph = pymorphy2.MorphAnalyzer()
c = Counter()

stop_words = stopwords.words('russian')

Функция, которая чистит отзывы от всех знаков препинания, заменяет числа

In [9]:
def clean_text(text):
    text = text.lower()
    regular = r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
    text = re.sub(regular, '', text)
    text = re.sub(r'(\d+\s\d+)|(\d+)',' NUM ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

Функция, которая лемматизирует отзыв, приводит его к нижнему регистру и токенизирует

In [10]:
def lemmatize(a):
    words = a.split() 
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

In [11]:
pos_clean = clean_text(text)
neg_clean = clean_text(text_1)

In [12]:
pos_lem = lemmatize(pos_clean)
neg_lem = lemmatize(neg_clean)

Убираем стоп-слова

In [13]:
list_pos = [word1 for word1 in pos_lem if word1 not in stop_words]
list_neg = [word2 for word2 in neg_lem if word2 not in stop_words]

In [14]:
review = []
for words in text_2:
    cleaner = clean_text(words)
    final = lemmatize(cleaner)
    review.append(final)

Считаем слова в позитивных и негативных отзывах

In [11]:
#fr = []
#for words_pos in list_pos:
    #a = c[words_pos]
    #fr.append(a)

In [12]:
#for words_pos in list_pos:
    #c[words_pos] += 1

Если слово употребляется меньше двух раз, то мы его не записываем

In [15]:
freq_pos = [item for item in set(list_pos) if list_pos.count(item) > 1]
freq_neg = [item for item in set(list_neg) if list_neg.count(item) > 1]

Далее мы сравниваем слова в списке позитивных и негативных. Если есть совпадающиеся слова, то они удаляются

In [16]:
new_freq_pos = []
for w in freq_pos:
    if w in freq_neg:
        continue
    else:
        new_freq_pos.append(w) 

In [17]:
new_freq_neg = []
for t in freq_neg:
    if t in freq_pos:
        continue
    else:
        new_freq_neg.append(t)

Получив список с позитивными словами и с негативными словами, мы определяем отзыв позитивный или негативный

In [18]:
for i in review:
    for j in i:
        if j in new_freq_neg:
            flag = True
            break
        else:
            flag = False
            break
            
    if flag:
        print('Это негативный отзыв')
    else:
        print('Если это не негативный отзыв, то скорее всего позитивный')

Если это не негативный отзыв, то скорее всего позитивный
Если это не негативный отзыв, то скорее всего позитивный
Если это не негативный отзыв, то скорее всего позитивный
Если это не негативный отзыв, то скорее всего позитивный
Если это не негативный отзыв, то скорее всего позитивный
Если это не негативный отзыв, то скорее всего позитивный
Это негативный отзыв
Если это не негативный отзыв, то скорее всего позитивный
Это негативный отзыв
Это негативный отзыв


Чтобы улучшить код, можно сделать более мелкую классификацию. Например, не только позитивный и негативный, а негативные более негативный, нейтральный, более позитивный, позитивный. Также можно учитывать отрицательную частицу не и такой знак препинания как ! и его количество. 